# Download MARBERT checkpoint

In [1]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz

--2021-02-24 19:37:31--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 34.201.172.85
Connecting to huggingface.co (huggingface.co)|34.201.172.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc [following]
--2021-02-24 19:37:31--  https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 52.84.169.48, 52.84.169.23, 52.84.169.2, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|52.84.169.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607066087 (579M) [application/x-gzip]
Saving to: ‘MARBERT_pytorch_verison.tar.gz’

MARBERT_pytorch_ver 100%[===================>] 578.94M   160MB/s    in 3.6s    

2021-02-24 19:

In [2]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

MARBERT_pytorch_verison/
MARBERT_pytorch_verison/pytorch_model.bin
MARBERT_pytorch_verison/config.json
MARBERT_pytorch_verison/vocab.txt


In [3]:
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv

--2021-02-24 19:37:43--  https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149495 (146K) [text/plain]
Saving to: ‘UBC_AJGT_final_shuffled_train.tsv’

UBC_AJGT_final_shuf 100%[===================>] 145.99K  --.-KB/s    in 0.02s   

2021-02-24 19:37:43 (9.10 MB/s) - ‘UBC_AJGT_final_shuffled_train.tsv’ saved [149495/149495]

--2021-02-24 19:37:43--  https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
H

In [4]:
!mkdir -p AJGT
!mv UBC_AJGT_final_shuffled_train.tsv ./AJGT/UBC_AJGT_final_shuffled_train.tsv
!mv UBC_AJGT_final_shuffled_test.tsv ./AJGT/UBC_AJGT_final_shuffled_test.tsv

In [5]:
!pip install GPUtil pytorch_pretrained_bert transformers

     |████████████████████████████████| 133kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 13.2MB/s 
     |████████████████████████████████| 133kB 51.1MB/s 
     |████████████████████████████████| 890kB 34.8MB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 
     |████████████████████████████████| 7.3MB 50.6MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=ae79aac7bd90a97dac6a54eafbe66ba64a9cc22418050753fc1ec7902e11d677
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f671e6c504617f5ed238dc3a87aacb363d7bad8c8c69d967cd1a1a964a5e22a2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built GPUtil sacremoses
ERROR: botocore 1.20.14 has requirement urllib3<

In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")

Mounted at /content/drive


# Fine-tuning code

In [7]:
# (1)load libraries 
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel


In [8]:
import re

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [10]:
def create_label2ind_file(file, label_col):
    labels_json = {}

    # load train_dev_test file

    df = pd.read_csv(file)
    df.loc[df.sarcasm==False,label_col]="Not_Sarcasm"
    df.loc[df.sarcasm==True,label_col]="Sarcasm"

    # get labels and sort it A-Z

    labels = df[label_col].unique()
    labels.sort()

    # convert labels to indexes

    for idx in range(0, len(labels)):
        labels_json[labels[idx]] = idx
    with open(label2idx_file, 'w') as json_file:
        json.dump(labels_json, json_file)

In [11]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    # longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [12]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


def data_prepare_BERT(
    file_path,
    lab2ind,
    tokenizer,
    content_col,
    label_col,
    MAX_LEN,
    ):

    # Use pandas to load dataset

    df = pd.read_csv(file_path)

    df = df[df[content_col].notnull()]
    df = df[df[label_col].notnull()]
    df[content_col] = df.tweet.apply(clean_text)
    df.loc[df.sarcasm==False,label_col]="Not_Sarcasm"
    df.loc[df.sarcasm==True,label_col]="Sarcasm"


    print ('Data size ', df.shape)

    # Create sentence and label lists

    sentences = df[content_col].values
    sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in
                 sentences]
    print ('The first sentence:')
    print (sentences[0])

    # Create sentence and label lists

    labels = df[label_col].values

    # print (labels)

    labels = [lab2ind[i] for i in labels]

    # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print ('Tokenize the first sentence:')
    print (tokenized_texts[0])

    # print("Label is ", labels[0])
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in
                 tokenized_texts]
    print ('Index numbers of the first sentence:')
    print (input_ids[0])

    # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
    # ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
    input_ids = pad_sequences(
        input_ids,
        maxlen=MAX_LEN + 2,
        dtype='long',
        truncating='post',
        padding='post',
        value=pad_ind,
        )
    print ('Index numbers of the first sentence after padding:\n',
           input_ids[0])

    # Create attention masks

    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding

    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert all of our data into torch tensors, the required datatype for our model

    inputs = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    masks = torch.tensor(attention_masks)
    return inputs, labels, masks


In [13]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


def data_prepare_BERT_test(
    file_path,
    lab2ind,
    tokenizer,
    content_col,
    
    MAX_LEN,
    ):

    # Use pandas to load dataset

    df = pd.read_csv(file_path)

    df = df[df[content_col].notnull()]
    df[content_col] = df.tweet.apply(clean_text)


  # df[content_col] = df.tweet.apply(clean_text)

    print ('Data size ', df.shape)

    # Create sentence and label lists

    sentences = df[content_col].values
    sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in
                 sentences]
    print ('The first sentence:')
    print (sentences[0])

    # Create sentence and label lists

   

    # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print ('Tokenize the first sentence:')
    print (tokenized_texts[0])

    # print("Label is ", labels[0])
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in
                 tokenized_texts]
    print ('Index numbers of the first sentence:')
    print (input_ids[0])

    # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
    # ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
    input_ids = pad_sequences(
        input_ids,
        maxlen=MAX_LEN + 2,
        dtype='long',
        truncating='post',
        padding='post',
        value=pad_ind,
        )
    print ('Index numbers of the first sentence after padding:\n',
           input_ids[0])

    # Create attention masks

    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding

    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert all of our data into torch tensors, the required datatype for our model

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)
    return inputs,  masks


In [14]:
# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#	  pred_flat = np.argmax(preds, axis=1).flatten()
#	  labels_flat = labels.flatten()
#	  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def flat_pred(preds, labels):
	pred_flat = np.argmax(preds, axis=1).flatten()
	labels_flat = labels.flatten()
	return pred_flat.tolist(), labels_flat.tolist()

In [15]:

def train(model, iterator, optimizer, scheduler, criterion):
	
	model.train()
	epoch_loss = 0
	for i, batch in enumerate(iterator):
		# Add batch to GPU
		batch = tuple(t.to(device) for t in batch)
		# Unpack the inputs from our dataloader
		input_ids, input_mask, labels = batch
		outputs = model(input_ids, input_mask, labels=labels)
		loss, logits = outputs[:2]
		# delete used variables to free GPU memory
		del batch, input_ids, input_mask, labels
		optimizer.zero_grad()
		if torch.cuda.device_count() == 1:
			loss.backward()
			epoch_loss += loss.cpu().item()
		else:
			loss.sum().backward()
			epoch_loss += loss.sum().cpu().item()
		optimizer.step()
		torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
		# optimizer.step()
		scheduler.step()
	# free GPU memory
	if device == 'cuda':
		torch.cuda.empty_cache()
	return epoch_loss / len(iterator)

In [16]:

def evaluate(model, iterator, criterion):
	model.eval()
	epoch_loss = 0
	all_pred=[]
	all_label = []
	with torch.no_grad():
		for i, batch in enumerate(iterator):
			# Add batch to GPU
			batch = tuple(t.to(device) for t in batch)
			# Unpack the inputs from our dataloader
			input_ids, input_mask, labels = batch
			outputs = model(input_ids, input_mask, labels=labels)
			loss, logits = outputs[:2]
			# delete used variables to free GPU memory
			del batch, input_ids, input_mask
			if torch.cuda.device_count() == 1:
				epoch_loss += loss.cpu().item()
			else:
				epoch_loss += loss.sum().cpu().item()
			# identify the predicted class for each example in the batch
			probabilities, predicted = torch.max(logits.cpu().data, 1)
			# put all the true labels and predictions to two lists
			all_pred.extend(predicted)
			all_label.extend(labels.cpu())
	accuracy = accuracy_score(all_label, all_pred)
	f1score = f1_score(all_label, all_pred, average='macro') 
	recall = recall_score(all_label, all_pred, average='macro')
	precision = precision_score(all_label, all_pred, average='macro')
	report = classification_report(all_label, all_pred)
	return (epoch_loss / len(iterator)), accuracy, f1score, recall, precision



In [17]:

def fine_tuning(config):
	#---------------------------------------
	print ("[INFO] step (1) load train_test config file")
	# config_file = open(config_file, 'r', encoding="utf8")
	# config = json.load(config_file)
	task_name = config["task_name"]
	content_col = config["content_col"]
	label_col = config["label_col"]
	train_file = config["data_dir"]+config["train_file"]
	dev_file = config["data_dir"]+config["dev_file"]
	sortby = config["sortby"]
	max_seq_length= int(config["max_seq_length"])
	batch_size = int(config["batch_size"])
	lr_var = float(config["lr"])
	model_path = config['pretrained_model_path']
	num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
	global label2idx_file
	label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
	#-------------------------------------------------------
	print ("[INFO] step (2) convert labels2index")
	create_label2ind_file(train_file, label_col)
	print (label2idx_file)
	#---------------------------------------------------------
	print ("[INFO] step (3) check checkpoit directory and report file")
	ckpt_dir = config["data_dir"]+task_name+"_bert_ckpt/"
	report = ckpt_dir+task_name+"_report.tsv"
	sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
	if not os.path.exists(ckpt_dir):
		os.mkdir(ckpt_dir)
	#-------------------------------------------------------
	print ("[INFO] step (4) load label to number dictionary")
	lab2ind = json.load(open(label2idx_file))
	print ("[INFO] train_file", train_file)
	print ("[INFO] dev_file", dev_file)
	print ("[INFO] num_epochs", num_epochs)
	print ("[INFO] model_path", model_path)
	print ("max_seq_length", max_seq_length, "batch_size", batch_size)
	#-------------------------------------------------------
	print ("[INFO] step (5) Use defined funtion to extract tokanize data")
	# tokenizer from pre-trained BERT model
	print ("loading BERT setting")
	tokenizer = BertTokenizer.from_pretrained(model_path)
	train_inputs, train_labels, train_masks = data_prepare_BERT(train_file, lab2ind, tokenizer,content_col, label_col, max_seq_length)
	validation_inputs, validation_labels, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
	# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
	model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(lab2ind))
	#--------------------------------------
	print ("[INFO] step (6) Create an iterator of data with torch DataLoader.")
#		  This helps save on memory during training because, unlike a for loop,\
#		  with an iterator the entire dataset does not need to be loaded into memory")
	train_data = TensorDataset(train_inputs, train_masks, train_labels)
	train_dataloader = DataLoader(train_data, batch_size=batch_size)
	#---------------------------
	validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
	validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
	#------------------------------------------
	print ("[INFO] step (7) run with parallel GPUs")
	if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
	else:
		print("Run", "with CPU")
		model = model
	#---------------------------------------------------
	print ("[INFO] step (8) set Parameters, schedules, and loss function")
	global max_grad_norm
	max_grad_norm = 1.0
	warmup_proportion = 0.1
	num_training_steps	= len(train_dataloader) * num_epochs
	num_warmup_steps = num_training_steps * warmup_proportion
	### In Transformers, optimizer and schedules are instantiated like this:
	# Note: AdamW is a class from the huggingface library
	# the 'W' stands for 'Weight Decay"
	optimizer = AdamW(model.parameters(), lr=lr_var, correct_bias=False)
	# schedules
	scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
	# We use nn.CrossEntropyLoss() as our loss function. 
	criterion = nn.CrossEntropyLoss()
	#---------------------------------------------------
	print ("[INFO] step (9) start fine_tuning")
	for epoch in trange(num_epochs, desc="Epoch"):
		train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)	  
		val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)
# 		print (train_loss, val_acc)
		# Create checkpoint at end of each epoch
		if not os.path.exists(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/'): os.mkdir(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/')
		model.save_pretrained(ckpt_dir+ 'model_' + str(int(epoch + 1)) + '/')
		epoch_eval_results = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
					  "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1,"lr":lr_var }
		with open(report,"a") as fOut:
			fOut.write(json.dumps(epoch_eval_results)+"\n")
			fOut.flush()
		#------------------------------------
		report_df = pd.read_json(report, orient='records', lines=True)
		report_df.sort_values(by=[sortby],ascending=False, inplace=True)
		report_df.to_csv(sorted_report,sep="\t",index=False)
	return report_df

# Run fine-tuning for 5 epochs

In [18]:

config={"task_name": "Ironic_MARBERT_Task_1", #output directory name
             "data_dir": "", #data directory
             "train_file": "", #train file path
             "dev_file": "", #dev file path or test file path
             "pretrained_model_path": '', #MARBERT checkpoint path
             "epochs": 5, #number of epochs
             "content_col": "tweet", #text column
             "label_col": "sarcasm", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 300, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df = fine_tuning(config)

[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index
/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/Ironic_MARBERT_Task_1_labels-dict.json
[INFO] step (3) check checkpoit directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file {'Not_Sarcasm': 0, 'Sarcasm': 1}
[INFO] dev_file /content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/ArSarcasm_test.csv
[INFO] num_epochs 5
[INFO] model_path /content/MARBERT_pytorch_verison
max_seq_length 300 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (12548, 4)
The first sentence:
[CLS] د محمود العلايلياري ان الفريق احمد شفيق رقم مهم في المعادله السياسيه المصريه ولا يمكن اغفالههل تري ان هذا صحيحاربعه زائد واحد [SEP]
Tokenize the first sentence:
['[CLS]', 'د', 'محمود', 'العلا', '##يلي', '##اري', 'ان', 'الفريق', 'احمد', 'شفيق', 'رقم', 'مهم', 'في', 'المعاد', '##له', 'السياسيه', 'المصريه', 'ولا', 'يمكن', 'اغف', '##اله', '##ه

Some weights of the model checkpoint at /content/MARBERT_pytorch_verison were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mode

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU
[INFO] step (8) set Parameters, schedules, and loss function
[INFO] step (9) start fine_tuning



Epoch: 100%|██████████| 5/5 [32:51<00:00, 394.32s/it]


In [ ]:
report_df.head(5)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
4,5,0.165897,0.944076,0.908273,0.892026,0.899867,0.000002
3,4,0.181331,0.942180,0.915302,0.883891,0.898519,0.000002
2,3,0.212130,0.929384,0.873839,0.869545,0.871671,0.000002
1,2,0.256924,0.917062,0.840822,0.851578,0.846057,0.000002
0,1,0.388817,0.886730,0.743404,0.808139,0.769423,0.000002


In [ ]:
def predict_extrnal_test_set():
    lab2ind = json.load(open()) # load label2index file generated and stored in traiing pahse
    tokenizer = BertTokenizer.from_pretrained('')#load bert tokenizer
    max_seq_length=300
    validation_inputs,  validation_masks = data_prepare_BERT_test('', lab2ind, tokenizer, 'tweet', max_seq_length)
    # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
    model = BertForSequenceClassification.from_pretrained('', num_labels=len(lab2ind))
    validation_data = TensorDataset(validation_inputs, validation_masks)
    validation_dataloader = DataLoader(validation_data, batch_size=32)
    ################ Evaluation and prediction#####################
    if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
    else:
      print("Run", "with CPU")
      model = model
    criterion = nn.CrossEntropyLoss()
    model.eval()
    all_pred=[]
    all_label = []
    with torch.no_grad():
      for i, batch in enumerate(validation_dataloader):    
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask = batch
        outputs = model(input_ids, input_mask)
        logits = outputs[:2]
        # delete used variables to free GPU memory
        del batch, input_ids, input_mask   
        # identify the predicted class for each example in the batch
        probabilities, predicted = torch.max(outputs[0], 1)
        # put all the true labels and predictions to two lists
        all_pred.extend(predicted)
    return all_pred
    

In [ ]:
all_pred = predict_extrnal_test_set()

In [ ]:
def write_output_file(all_pred):

    f = open("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/Task1_MaRBERT.txt", "w")
    id=1
    for pred in all_pred:

      if pred==0:
        f.write("FALSE\n")
      else:
        f.write("TRUE\n")
    f.close()


In [ ]:
write_output_file(all_pred)